<a href="https://colab.research.google.com/github/pksX01/Python-Tutorials/blob/master/rdd_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Transformations : ** map, flatMap, filter, groupBy, groupByKey, reduceByKey, union, sortByKey, join, coealesce, repartition, mapValues, aggregateByKey etc. </br>
**Actions : ** first, take, collect, reduce, countByKey, saveAsTextFile, count, foreach etc.

In [ ]:
import pyspark

In [ ]:
from pyspark import SparkContext

In [ ]:
sc = SparkContext()

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-3352225748674864> in <module> 
 ----> 1 sc = SparkContext ( ) 

 /databricks/spark/python/pyspark/context.py in __init__ (self, master, appName, sparkHome, pyFiles, environment, batchSize, serializer, conf, gateway, jsc, profiler_cls) 
 143 " is not allowed as it is a security risk.")
 144 
 --> 145 SparkContext . _ensure_initialized ( self , gateway = gateway , conf = conf ) 
 146 try : 
 147 self._do_init(master, appName, sparkHome, pyFiles, environment, batchSize, serializer,

 /databricks/spark/python/pyspark/context.py in _ensure_initialized (cls, instance, gateway, conf) 
 348 
 349 # Raise error if there is already a running Spark context 
 --> 350 raise ValueError(
 351 "Cannot run multiple SparkContexts at once; " 
 352 "existing SparkContext(app=%s, master=%s)" 

 ValueError : Cannot run multiple SparkContexts at once; existing SparkContext(app=Databricks Shell, master=spark://driver-proxy:7077) created by __init__ at /databricks/python_shell/scripts/PythonShellImpl.py:1887

In [ ]:
sc

Out[4]:

SparkContext 

 Spark UI 

 
 Version 
 v3.1.1 
 Master 
 spark://driver-proxy:7077 
 AppName 
 Databricks Shell

**Reading a file**

In [ ]:
rdd_text = sc.textFile('/FileStore/tables/healthcare_dataset_stroke_data.csv')
rdd_text

Out[2]: /FileStore/tables/healthcare_dataset_stroke_data.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [ ]:
rdd_text.count()

Out[6]: 5111

In [ ]:
rdd_text.first()

Out[7]: 'id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke'

In [ ]:
rdd_text.take(5)

Out[8]: ['id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke',
 '9046,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1',
 '51676,Female,61,0,0,Yes,Self-employed,Rural,202.21,N/A,never smoked,1',
 '31112,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1',
 '60182,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1']

In [ ]:
for line in rdd_text.take(5):
  print(line)

id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
9046,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61,0,0,Yes,Self-employed,Rural,202.21,N/A,never smoked,1
31112,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1

**Removing duplicates records**

In [ ]:
rdd_text_unique = rdd_text.distinct()

In [ ]:
rdd_text_unique.count()

Out[11]: 5111

**Filtering only data where stroke = '1' and saving that data as CSV**

In [ ]:
positive_stroke_data = rdd_text.map(lambda line: line.split(',')).filter(lambda x: x[-1]=='1')

In [ ]:
positive_stroke_data.count()

Out[33]: 249

In [ ]:
positive_stroke_data.first()

Out[34]: ['9046',
 'Male',
 '67',
 '0',
 '1',
 'Yes',
 'Private',
 'Urban',
 '228.69',
 '36.6',
 'formerly smoked',
 '1']

In [ ]:
#retreiving header
header = sc.parallelize([rdd_text.map(lambda line: line.split()).first()])

In [ ]:
header.collect()

Out[69]: [['id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke']]

In [ ]:
#combining header and data having stroke = '1'
positive_stroke_data_with_header = header.union(positive_stroke_data)

In [ ]:
positive_stroke_data_with_header.count()

Out[71]: 250

In [ ]:
positive_stroke_data_with_header.take(2)

Out[72]: [['id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke'],
 ['9046',
 'Male',
 '67',
 '0',
 '1',
 'Yes',
 'Private',
 'Urban',
 '228.69',
 '36.6',
 'formerly smoked',
 '1']]

In [ ]:
#Saving as csv file to Databricks File System (DBFS)
positive_stroke_data_with_header.saveAsTextFile('/FileStore/savedData/positive_stroke_data.csv')

In [ ]:
from pyspark import SQLContext
sqlCon = SQLContext(sc)

In [ ]:
sqlCon.read.csv('/FileStore/savedData/positive_stroke_data.csv').tail(1)

Out[79]: [Row(_c0="['id", _c1='gender', _c2='age', _c3='hypertension', _c4='heart_disease', _c5='ever_married', _c6='work_type', _c7='Residence_type', _c8='avg_glucose_level', _c9='bmi', _c10='smoking_status', _c11="stroke']")]

**Finding different values of Residence_type and their count in original data**

In [ ]:
def split_lines(line):
  return line.split(',')

In [ ]:
rdd_text.map(split_lines).map(lambda values: (values[7], 1)).reduceByKey(lambda a,b : a+b).collect()

Out[4]: [('Residence_type', 1), ('Rural', 2514), ('Urban', 2596)]

**Find out the number of persons having stroke for each gender**

In [ ]:
#1st method (reduceByKey() and sortByKey() are transformations, hence collect() is required to get output)
rdd_text.map(split_lines).map(lambda x: ((x[1], x[-1]), 1)).reduceByKey(lambda a,b : a+b).sortByKey().collect()

Out[20]: [(('Female', '0'), 2853),
 (('Female', '1'), 141),
 (('Male', '0'), 2007),
 (('Male', '1'), 108),
 (('Other', '0'), 1),
 (('gender', 'stroke'), 1)]

In [ ]:
#2nd method (countByKey() is action, hence collect() will not be used but sortByKey() is a transformation. hence it can't be used after countByKey()). It returns hash map.
rdd_text.map(split_lines).map(lambda x: ((x[1], x[-1]), 1)).sortByKey().countByKey()

Out[27]: defaultdict(int,
 {('Female', '0'): 2853,
 ('Female', '1'): 141,
 ('Male', '0'): 2007,
 ('Male', '1'): 108,
 ('Other', '0'): 1,
 ('gender', 'stroke'): 1})

**Grouping data on the basis of gender**</br>
groupBy generates data in the form of key and value pair. Key will be the value on which basis grouping is being done and whole data related to that key will be in value of tuple. To retrive the values, they need to be converted into list.

In [ ]:
rdd_grouped = rdd_text.map(lambda line: line.split(',')).groupBy(lambda x: x[1])

In [ ]:
rdd_grouped.take(2)

Out[10]: [('gender', <pyspark.resultiterable.ResultIterable at 0x7f7fc89676d0>),
 ('Male', <pyspark.resultiterable.ResultIterable at 0x7f7fc8967820>)]

In [ ]:
print([(key, list(val)) for (key, val) in rdd_grouped.take(4)])

[('Female', [['72369', 'Female', '14', '0', '0', 'No', 'children', 'Rural', '65.41', '19.5', 'Unknown', '0'], ['7171', 'Female', '56', '0', '0', 'Yes', 'Govt_job', 'Urban', '102.51', '55.7', 'Unknown', '0'], ['507', 'Female', '28', '0', '0', 'Yes', 'Private', 'Rural', '94.15', '23.1', 'smokes', '0'], ['54072', 'Female', '11', '0', '0', 'No', 'children', 'Urban', '81.31', '18.8', 'never smoked', '0'], ['49760', 'Female', '63', '0', '0', 'Yes', 'Private', 'Rural', '78.96', '28.6', 'never smoked', '0'], ['61821', 'Female', '59', '0', '0', 'Yes', 'Private', 'Rural', '123.47', '27.5', 'Unknown', '0'], ['6174', 'Female', '35', '0', '0', 'No', 'Private', 'Urban', '71.59', '40.3', 'never smoked', '0'], ['65564', 'Female', '48', '0', '0', 'Yes', 'Private', 'Urban', '57.43', '53.5', 'formerly smoked', '0'], ['67667', 'Female', '72', '1', '0', 'Yes', 'Self-employed', 'Rural', '112.12', '30.5', 'never smoked', '0'], ['50363', 'Female', '73', '1', '0', 'Yes', 'Private', 'Rural', '60.98', '29.9', 'formerly smoked', '0'], ['4740', 'Female', '24', '0', '0', 'No', 'Private', 'Urban', '86.35', '32.7', 'never smoked', '0'], ['63457', 'Female', '78', '0', '1', 'No', 'Self-employed', 'Urban', '110.78', '22.9', 'never smoked', '0'], ['18595', 'Female', '77', '0', '0', 'Yes', 'Private', 'Urban', '99.78', '38', 'never smoked', '0'], ['64912', 'Female', '59', '0', '0', 'Yes', 'Self-employed', 'Rural', '201.45', '43.8', 'smokes', '0'], ['29872', 'Female', '35', '0', '0', 'Yes', 'Private', 'Urban', '83.89', '25.5', 'never smoked', '0'], ['67243', 'Female', '75', '0', '1', 'Yes', 'Private', 'Urban', '206.15', '25.4', 'never smoked', '0'], ['3494', 'Female', '80', '0', '0', 'Yes', 'Private', 'Rural', '102.9', '26.7', 'Unknown', '0'], ['37307', 'Female', '35', '0', '0', 'Yes', 'Private', 'Urban', '65.48', '50.5', 'never smoked', '0'], ['41175', 'Female', '22', '0', '0', 'No', 'Govt_job', 'Urban', '123.23', '21.3', 'Unknown', '0'], ['31402', 'Female', '62', '0', '0', 'Yes', 'Self-employed', 'Rural', '102.21', '36.3', 'never smoked', '0'], ['18996', 'Female', '13', '0', '0', 'No', 'children', 'Urban', '105.22', '18.4', 'Unknown', '0'], ['63193', 'Female', '44', '0', '0', 'Yes', 'Private', 'Rural', '88.75', '25.6', 'Unknown', '0'], ['58107', 'Female', '59', '0', '0', 'Yes', 'Private', 'Rural', '79.18', '30', 'Unknown', '0'], ['28647', 'Female', '35', '0', '0', 'Yes', 'Private', 'Urban', '81.33', '28.9', 'never smoked', '0'], ['57086', 'Female', '52', '0', '0', 'Yes', 'Private', 'Urban', '126.68', '28.1', 'never smoked', '0'], ['5505', 'Female', '76', '0', '0', 'Yes', 'Private', 'Urban', '196.61', '23', 'never smoked', '0'], ['44112', 'Female', '51', '0', '0', 'Yes', 'Self-employed', 'Urban', '219.92', '33.5', 'formerly smoked', '0'], ['56645', 'Female', '79', '0', '0', 'Yes', 'Govt_job', 'Rural', '79.16', '34.8', 'formerly smoked', '0'], ['16652', 'Female', '69', '0', '0', 'Yes', 'Self-employed', 'Urban', '99.68', '17.6', 'formerly smoked', '0'], ['32445', 'Female', '78', '0', '0', 'Yes', 'Self-employed', 'Urban', '79.55', '21.1', 'formerly smoked', '0'], ['67112', 'Female', '56', '0', '0', 'Yes', 'Private', 'Rural', '77.66', '40.8', 'never smoked', '0'], ['35022', 'Female', '69', '0', '0', 'Yes', 'Private', 'Urban', '111.48', '37', 'smokes', '0'], ['21625', 'Female', '25', '0', '0', 'Yes', 'Private', 'Urban', '84.25', '24.5', 'Unknown', '0'], ['364', 'Female', '58', '0', '0', 'Yes', 'Private', 'Urban', '105.74', '26.8', 'formerly smoked', '0'], ['59669', 'Female', '28', '0', '0', 'Yes', 'Private', 'Rural', '58.41', '21', 'Unknown', '0'], ['69900', 'Female', '46', '0', '0', 'Yes', 'Govt_job', 'Urban', '56.89', '23.8', 'smokes', '0'], ['10273', 'Female', '37', '0', '0', 'Yes', 'Private', 'Rural', '86.49', '24.4', 'Unknown', '0'], ['587', 'Female', '14', '0', '0', 'No', 'children', 'Rural', '92.22', '22.8', 'Unknown', '0'], ['55856', 'Female', '60', '0', '0', 'Yes', 'Private', 'Rural', '83.16', '29.7', 'smokes', '0'], ['67724', 'Female', '65', '0', '0', 'Yes', 

In [ ]:
for key, val in rdd_grouped.collect():
  print((key, list(val)))

('Female', [['72369', 'Female', '14', '0', '0', 'No', 'children', 'Rural', '65.41', '19.5', 'Unknown', '0'], ['7171', 'Female', '56', '0', '0', 'Yes', 'Govt_job', 'Urban', '102.51', '55.7', 'Unknown', '0'], ['507', 'Female', '28', '0', '0', 'Yes', 'Private', 'Rural', '94.15', '23.1', 'smokes', '0'], ['54072', 'Female', '11', '0', '0', 'No', 'children', 'Urban', '81.31', '18.8', 'never smoked', '0'], ['49760', 'Female', '63', '0', '0', 'Yes', 'Private', 'Rural', '78.96', '28.6', 'never smoked', '0'], ['61821', 'Female', '59', '0', '0', 'Yes', 'Private', 'Rural', '123.47', '27.5', 'Unknown', '0'], ['6174', 'Female', '35', '0', '0', 'No', 'Private', 'Urban', '71.59', '40.3', 'never smoked', '0'], ['65564', 'Female', '48', '0', '0', 'Yes', 'Private', 'Urban', '57.43', '53.5', 'formerly smoked', '0'], ['67667', 'Female', '72', '1', '0', 'Yes', 'Self-employed', 'Rural', '112.12', '30.5', 'never smoked', '0'], ['50363', 'Female', '73', '1', '0', 'Yes', 'Private', 'Rural', '60.98', '29.9', 'formerly smoked', '0'], ['4740', 'Female', '24', '0', '0', 'No', 'Private', 'Urban', '86.35', '32.7', 'never smoked', '0'], ['63457', 'Female', '78', '0', '1', 'No', 'Self-employed', 'Urban', '110.78', '22.9', 'never smoked', '0'], ['18595', 'Female', '77', '0', '0', 'Yes', 'Private', 'Urban', '99.78', '38', 'never smoked', '0'], ['64912', 'Female', '59', '0', '0', 'Yes', 'Self-employed', 'Rural', '201.45', '43.8', 'smokes', '0'], ['29872', 'Female', '35', '0', '0', 'Yes', 'Private', 'Urban', '83.89', '25.5', 'never smoked', '0'], ['67243', 'Female', '75', '0', '1', 'Yes', 'Private', 'Urban', '206.15', '25.4', 'never smoked', '0'], ['3494', 'Female', '80', '0', '0', 'Yes', 'Private', 'Rural', '102.9', '26.7', 'Unknown', '0'], ['37307', 'Female', '35', '0', '0', 'Yes', 'Private', 'Urban', '65.48', '50.5', 'never smoked', '0'], ['41175', 'Female', '22', '0', '0', 'No', 'Govt_job', 'Urban', '123.23', '21.3', 'Unknown', '0'], ['31402', 'Female', '62', '0', '0', 'Yes', 'Self-employed', 'Rural', '102.21', '36.3', 'never smoked', '0'], ['18996', 'Female', '13', '0', '0', 'No', 'children', 'Urban', '105.22', '18.4', 'Unknown', '0'], ['63193', 'Female', '44', '0', '0', 'Yes', 'Private', 'Rural', '88.75', '25.6', 'Unknown', '0'], ['58107', 'Female', '59', '0', '0', 'Yes', 'Private', 'Rural', '79.18', '30', 'Unknown', '0'], ['28647', 'Female', '35', '0', '0', 'Yes', 'Private', 'Urban', '81.33', '28.9', 'never smoked', '0'], ['57086', 'Female', '52', '0', '0', 'Yes', 'Private', 'Urban', '126.68', '28.1', 'never smoked', '0'], ['5505', 'Female', '76', '0', '0', 'Yes', 'Private', 'Urban', '196.61', '23', 'never smoked', '0'], ['44112', 'Female', '51', '0', '0', 'Yes', 'Self-employed', 'Urban', '219.92', '33.5', 'formerly smoked', '0'], ['56645', 'Female', '79', '0', '0', 'Yes', 'Govt_job', 'Rural', '79.16', '34.8', 'formerly smoked', '0'], ['16652', 'Female', '69', '0', '0', 'Yes', 'Self-employed', 'Urban', '99.68', '17.6', 'formerly smoked', '0'], ['32445', 'Female', '78', '0', '0', 'Yes', 'Self-employed', 'Urban', '79.55', '21.1', 'formerly smoked', '0'], ['67112', 'Female', '56', '0', '0', 'Yes', 'Private', 'Rural', '77.66', '40.8', 'never smoked', '0'], ['35022', 'Female', '69', '0', '0', 'Yes', 'Private', 'Urban', '111.48', '37', 'smokes', '0'], ['21625', 'Female', '25', '0', '0', 'Yes', 'Private', 'Urban', '84.25', '24.5', 'Unknown', '0'], ['364', 'Female', '58', '0', '0', 'Yes', 'Private', 'Urban', '105.74', '26.8', 'formerly smoked', '0'], ['59669', 'Female', '28', '0', '0', 'Yes', 'Private', 'Rural', '58.41', '21', 'Unknown', '0'], ['69900', 'Female', '46', '0', '0', 'Yes', 'Govt_job', 'Urban', '56.89', '23.8', 'smokes', '0'], ['10273', 'Female', '37', '0', '0', 'Yes', 'Private', 'Rural', '86.49', '24.4', 'Unknown', '0'], ['587', 'Female', '14', '0', '0', 'No', 'children', 'Rural', '92.22', '22.8', 'Unknown', '0'], ['55856', 'Female', '60', '0', '0', 'Yes', 'Private', 'Rural', '83.16', '29.7', 'smokes', '0'], ['67724', 'Female', '65', '0', '0', 'Yes', '

**Finding no of persons of each gender**</br>
groupByKey generates data in the form of key and value pair. Key will be the value on which basis grouping is being done and whole data related to that key will be in value of tuple. To retrive the values, they need to be converted into list.

In [ ]:
rdd_groupedByKey = rdd_text.map(lambda line: line.split(',')).map(lambda x: (x[1],1)).groupByKey()

In [ ]:
rdd_groupedByKey.take(2)

Out[11]: [('gender', <pyspark.resultiterable.ResultIterable at 0x7f7fc88ba1c0>),
 ('Male', <pyspark.resultiterable.ResultIterable at 0x7f7fc88baa00>)]

In [ ]:
print([x[0], list(x[1])] for x in rdd_groupedByKey.collect())

<generator object <genexpr> at 0x7f7fc892a7b0>

In [ ]:
for x in rdd_groupedByKey.collect():
  print((x[0], list(x[1])))

('gender', [1])
('Male', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
rdd_groupedByKey.mapValues(sum).sortByKey().collect()

Out[9]: [('Female', 2994), ('Male', 2115), ('Other', 1), ('gender', 1)]

**map vs flatMap**

In [ ]:
rdd_text.flatMap(split_lines).take(20)

Out[18]: ['id',
 'gender',
 'age',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'avg_glucose_level',
 'bmi',
 'smoking_status',
 'stroke',
 '9046',
 'Male',
 '67',
 '0',
 '1',
 'Yes',
 'Private',
 'Urban']

In [ ]:
rdd_text.map(split_lines).take(2)

Out[20]: [['id',
 'gender',
 'age',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'avg_glucose_level',
 'bmi',
 'smoking_status',
 'stroke'],
 ['9046',
 'Male',
 '67',
 '0',
 '1',
 'Yes',
 'Private',
 'Urban',
 '228.69',
 '36.6',
 'formerly smoked',
 '1']]